In [1]:
import pandas as pd
from SentenceParser import SentenceParser
from PrepareSentenceContext import PrepareSentenceContext
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from PerplexityEvaluator import PerplexityEvaluator
from PrepareSentenceContext import PrepareSentenceContext
from glob import glob
import logging
import torch
from tqdm import tqdm
import traceback
from many_atomic_detections import generate_responses
from itertools import product

/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py:37: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub("(</?[a-zA-Z0-9 ]+>)\s+", r"\1. ", text)  # to make sure that tags are in separate sentences


In [2]:
dataset_paths = ['src/wiki_dataset.csv']
model_names = ['openai-community/gpt2']
context_policies = ['previous-3-sentences']
authors = ['human_text', 'Falcon']

In [3]:
for ds_path, model_name, context_policy, author in product(dataset_paths, model_names, context_policies, authors):
    generate_responses(ds_path, model_name, context_policy, author)

INFO:root:Processing wiki dataset...
INFO:root:Iterating over texts...


Saving results to Responses/wiki_human_text_previous-3-sentences_gpt2.csv


Processing texts:   0%|          | 0/10 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 57, in iterate_over_texts
    r = process_text(row[author], atomic_detector, parser)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 28, in process_text
    chunks = parser(text)
             ^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 26, in __call__
    return self.parse_sentences(text)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 65, in parse_sentences
    lengths.append(len(sent_text.split()))
                       ^^^^^^^^^
UnboundLocalError: cannot access local variable 'sent_text' where it is not as

Error processing 26709147
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 4508056
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 50875
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 36670656
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 11907622
Error details: cannot access local variable 'sent_text' where it is not associated with a value


Traceback (most recent call last):
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 57, in iterate_over_texts
    r = process_text(row[author], atomic_detector, parser)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 28, in process_text
    chunks = parser(text)
             ^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 26, in __call__
    return self.parse_sentences(text)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 65, in parse_sentences
    lengths.append(len(sent_text.split()))
                       ^^^^^^^^^
UnboundLocalError: cannot access local variable 'sent_text' where it is not associated with a value
Traceback (most recent call last):

...

Error processing 26709147
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 4508056
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 50875
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 36670656
Error details: cannot access local variable 'sent_text' where it is not associated with a value


Traceback (most recent call last):
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 57, in iterate_over_texts
    r = process_text(row[author], atomic_detector, parser)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 28, in process_text
    chunks = parser(text)
             ^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 26, in __call__
    return self.parse_sentences(text)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 65, in parse_sentences
    lengths.append(len(sent_text.split()))
                       ^^^^^^^^^
UnboundLocalError: cannot access local variable 'sent_text' where it is not associated with a value
Traceback (most recent call last):

Error processing 11907622
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 4352382
Error details: cannot access local variable 'sent_text' where it is not associated with a value


Traceback (most recent call last):
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 57, in iterate_over_texts
    r = process_text(row[author], atomic_detector, parser)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/many_atomic_detections.py", line 28, in process_text
    chunks = parser(text)
             ^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 26, in __call__
    return self.parse_sentences(text)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamvinestock/Documents/GitHub/NLP-Authorship-Attribution/PrepareSentenceContext.py", line 65, in parse_sentences
    lengths.append(len(sent_text.split()))
                       ^^^^^^^^^
UnboundLocalError: cannot access local variable 'sent_text' where it is not associated with a value
Traceback (most recent call last):

Error processing 31533973
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 758725
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 39905881
Error details: cannot access local variable 'sent_text' where it is not associated with a value
Error processing 52727235
Error details: cannot access local variable 'sent_text' where it is not associated with a value
